In [1]:
# https://blog.csdn.net/qq_14845119/article/details/79028094
## 原文：https://blog.csdn.net/ali197294332/article/details/78720309 

import os
import tensorflow as tf
## 对于灰度图，PIL读出是二维的矩阵
#from PIL import Image
## cv对任何图像都会按照RGB的方式读取
import cv2

In [4]:
'''
tf.python_io.TFRecordWriter返回一个writer对象用于将data_dir制作后的数据存入filepath中保存，
该文件就是tfrecords文件。另外，tf.train.Example将数据处理成key-value(在这里就是标签-图像)的
格式返回一个example对象。最后writer将数据写到filepath中，关闭writer，
就完成了图片数据到二进制文件的制作过程。
'''

data_path = 'datasets/Mnist/'
train_tfrecords_dir = 'datasets/Mnist/train.tfrecords'
test_tfrecords_dir = 'datasets/Mnist/test.tfrecords'
num_classes = [str(i) for i in range(0,10)]


# data to int64List
def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value=[value]))
# data to floatlist
def _float_feature(value):
    return tf.train.Feature(float_list = tf.train.FloatList(value=[value]))
# data to byteslist
def _bytes_feature(value):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value=[value]))


# convert image data to tfrecords
def generate_tfrecords(data_dir, filepath):
    # gen a tfrecords object writer
    writer = tf.python_io.TFRecordWriter(filepath)
    
    for index, name in enumerate(num_classes):
        file_dir = data_dir + name + '/'
        for img_name in os.listdir(file_dir):
            img_path = file_dir + img_name
            img = cv2.imread(img_path)
            img_raw = img.tobytes()
            example = tf.train.Example(features=tf.train.Features(feature={
                'label': _int64_feature(index),
                'img_raw': _bytes_feature(img_raw)
            }))
            # covert example to binary string
            # for every image!!
            writer.write(example.SerializeToString())
    
    writer.close()
    
def generate_data():
    train_data_path = data_path + 'mnist_train/'
    test_data_path = data_path + 'mnist_test/'
    
    generate_tfrecords(train_data_path, train_tfrecords_dir)
    generate_tfrecords(test_data_path, test_tfrecords_dir)


In [5]:
generate_data()

In [6]:
'''
制作完成之后，在神经网络中如何读取和解析呢？如下代码
其中，tf.train.string_input_producer([filename])是将filename的文件内容制作成一个队列，
然后tf.parse_single_example按照固定的格式将内容解析出来，稍加处理即可得到label和img，
当然[filename]中可以有很多file，因为当图片数据太大时可能会将数据分成好几个部分
分别制作tfrecords进行存储和读取。
'''
def read_and_decode_tfrecored(filename):
    # produce file queue
    filename_dequeue = tf.train.string_input_producer([filename])
        
    # gen readere object
    reader = tf.TFRecordReader()
    # read data from filename_queue
    _, serialized_example = reader.read(filename_dequeue)
        
    # decode
    features = tf.parse_single_example(serialized_example,
                                        features={
                                            'label': tf.FixedLenFeature([], tf.int64),
                                            'img_raw': tf.FixedLenFeature([], tf.string)
                                        })
    
    label = tf.cast(features['label'], tf.int32)
    
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img,[28,28,3])
    img = tf.cast(img,tf.float32)/255. - 0.5
        
    return label, img

In [7]:
'''
tf.contrib.layers.xavier_initializer_conv2d()是对参数进行初始化的，
据某位童鞋的博客说，当激活函数是sigmoid或tanh时，这个初始化方法比较好，
但是当激活函数是relu时，使用tf.contrib.layers.variance_scaling_initializer比较好，
另外tf.contrib.layers.xavier_initializer()也是一种权值初始化方式。
而在神经网络中，权值的初始化非常重要，可以按照某种特定的分布来初始化，
以后可以尝试使用其他初始化方式从而加快收敛速度和准确率。
'''

# set parameter
epoch = 10000 # step
batch_size = 2000 # 100


## create network

class network(object):
    # define parameters w and b
    def __init__(self):
        with tf.variable_scope('weight'):
            self.weights = {
                'conv1': tf.get_variable('conv1', [5,5,3,32], 
                                         initializer=tf.contrib.layers.xavier_initializer_conv2d()),
                'conv2': tf.get_variable('conv2', [5,5,32,64], 
                                         initializer=tf.contrib.layers.xavier_initializer_conv2d()),
                'fc1': tf.get_variable('fc1', [7*7*64,1024], 
                                         initializer=tf.contrib.layers.xavier_initializer()),
                'fc2': tf.get_variable('fc2', [1024,10], 
                                         initializer=tf.contrib.layers.xavier_initializer())
            }
        with tf.variable_scope('bias'):
            self.biases = {
                'conv1': tf.get_variable('conv1', [32,],
                                         initializer=tf.constant_initializer(value=0.0, dtype=tf.float32)),
                'conv2': tf.get_variable('conv2', [64,],
                                         initializer=tf.constant_initializer(value=0.0, dtype=tf.float32)),
                'fc1': tf.get_variable('fc1', [1024,],
                                         initializer=tf.constant_initializer(value=0.0, dtype=tf.float32)),
                'fc2': tf.get_variable('fc2', [10,],
                                         initializer=tf.constant_initializer(value=0.0, dtype=tf.float32))
            }
    
    # define model
    def model(self,img):
        conv1 = tf.nn.bias_add(tf.nn.conv2d(img, self.weights['conv1'], strides=[1,1,1,1], padding='SAME'),
                              self.biases['conv1'])
        relu1 = tf.nn.relu(conv1)
        pool1 = tf.nn.max_pool(relu1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
        
        conv2 = tf.nn.bias_add(tf.nn.conv2d(pool1, self.weights['conv2'], strides=[1,1,1,1], padding='SAME'),
                              self.biases['conv2'])
        relu2 = tf.nn.relu(conv2)
        pool2 = tf.nn.max_pool(relu2, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
        
        flatten = tf.reshape(pool2, [-1, self.weights['fc1'].get_shape().as_list()[0]])
        
        drop1 = tf.nn.dropout(flatten, 0.8)
        
        fc1 = tf.matmul(drop1, self.weights['fc1']) + self.biases['fc1']
        fc1_relu = tf.nn.relu(fc1)
        
        fc2 = tf.matmul(fc1_relu, self.weights['fc2']) + self.biases['fc2']
        
        return fc2
    
    # define model test
    def test(self, img):
        img = tf.reshape(img, shape=[-1,28,28,3])
        
        conv1 = tf.nn.bias_add(tf.nn.conv2d(img, self.weights['conv1'], strides=[1,1,1,1], padding='SAME'),
                              self.biases['conv1'])
        relu1 = tf.nn.relu(conv1)
        pool1 = tf.nn.max_pool(relu1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
        
        conv2 = tf.nn.bias_add(tf.nn.conv2d(pool1, self.weights['conv2'], strides=[1,1,1,1], padding='SAME'),
                              self.biases['conv2'])
        relu2 = tf.nn.relu(conv2)
        pool2 = tf.nn.max_pool(relu2, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
        
        flatten = tf.reshape(pool2, [-1, self.weights['fc1'].get_shape().as_list()[0]])
        
        drop1 = tf.nn.dropout(flatten, 1)
        
        fc1 = tf.matmul(drop1, self.weights['fc1']) + self.biases['fc1']
        fc1_relu = tf.nn.relu(fc1)
        
        fc2 = tf.matmul(fc1_relu, self.weights['fc2']) + self.biases['fc2']
        
        return fc2
    
    # loss
    def softmax_loss(self, predicts, labels):
        predicts = tf.nn.softmax(predicts)
        labels = tf.one_hot(labels, len(num_classes))
        loss = -tf.reduce_mean(labels*tf.log(predicts))
        
        self.cost = loss
        
        return self.cost
    
    # optimizer
    def optimizer(self, loss, lr=0.001):
        train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)
        
        return train_op
    


In [8]:
'''
 tf.train.shuffle_batch是将队列里的数据打乱顺序使用n_threads个线程，
 batch_size大小的形式读取出来，capacity是整个队列的容量，min_after_deque代表参与顺序打乱的程度，
 参数越大代表数据越混乱。在本代码中，由于各个类别已经分好，大概都是5000张，而在制作tfrecords的时候是按顺序存储的，
 所以使用tf.train.shuffle_batch来打乱顺序，但是如果batch_size设置太小，
 那很大概率上每个batch_size的图像数据的类别都是一样的，造成过拟合，所以本次将batch_size设置成2000，
 这样效果比较明显，设置成1000也可以，或者在处理数据的时候提前将数据打乱，或者有其他方法欢迎下方讨论。
'''

def train():
    label, img = read_and_decode_tfrecored(train_tfrecords_dir)
    img_batch, label_batch = tf.train.shuffle_batch([img,label], 
                                             num_threads=16, batch_size=batch_size,
                                             capacity = 50000, min_after_dequeue=49000)
    net = network()
    predicts = net.model(img_batch)
    loss = net.softmax_loss(predicts, label_batch)
    opti = net.optimizer(loss)
    
    # add trace
    tf.summary.scalar('cost_function', loss)
    merged_summary_op = tf.summary.merge_all()
    
    train_correct = tf.equal(tf.cast(tf.argmax(predicts, 1), tf.int32), label_batch)
    train_accuracy = tf.reduce_mean(tf.cast(train_correct, tf.float32))
    
    # evaluate
    test_label, test_img = read_and_decode_tfrecored(test_tfrecords_dir)
    test_img_batch, test_label_batch = tf.train.shuffle_batch([test_img, test_label],
                                                             num_threads=16, batch_size=batch_size,
                                                             capacity=50000, min_after_dequeue=40000)
    test_out = net.test(test_img_batch)
    test_correct = tf.equal(tf.cast(tf.argmax(test_out,1), tf.int32), test_label_batch)
    test_accuracy = tf.reduce_mean(tf.cast(test_correct, tf.float32))
    
    
    # init variables
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        # manage different threads
        coord = tf.train.Coordinator() # #创建一个协调器，用于管理线程，发生错误时及时关闭线程
        summary_writer = tf.summary.FileWriter('log', sess.graph)
        
        # run deque
        # #各个线程开始读取数据，这一句如果没有，整个网络将被挂起
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        model_path = data_path + 'model.ckpt'
        
        # 模型恢复
        try:
            print("try to reload model...")
            tf.train.Saver(max_to_keep=None).restore(sess, model_path)
            print("reload sucessful...")
        except:
            print("reload model failed...")
        finally:
            print("training...")
        
        for i in range(1, epoch+1):
            if i%50 == 0:
                loss_np, _, label_np, img_np, predict_np = sess.run(
                    [loss, opti, label_batch, img_batch, predicts])
                tr_accuracy_np = sess.run([train_accuracy])
                print(i, "epch loss: ", loss_np, "   train accuracy: ", tr_accuracy_np)
                
            if i%200 == 0:
                summary_str, _l, _o = sess.run([merged_summary_op, loss, opti])
                summary_writer.add_summary(summary_str, i)
                te_acccracy = sess.run([test_accuracy])
                print("test accuracy: ", te_acccracy)
            
            # 模型保存，max_to_keep=None这个参数是保存最新的或者加载最新的模型
            if i%1000 == 0:
                tf.train.Saver(max_to_keep=None).save(sess, os.path.join(data_path, 'model.ckpt'))
        
        # 某个线程数据读取完或发生错误请求停止
        coord.request_stop()
        
        # #所有线程都请求停止后关闭线程
        coord.join(threads)

In [9]:
train()

try to reload model...
INFO:tensorflow:Restoring parameters from datasets/Mnist/model.ckpt
reload model failed...
training...
50 epch loss:  0.23117173    train accuracy:  [0.1095]
100 epch loss:  0.23174207    train accuracy:  [0.092]
150 epch loss:  0.23199707    train accuracy:  [0.093]
200 epch loss:  0.23159732    train accuracy:  [0.093]
test accuracy:  [0.1245]
250 epch loss:  0.2315979    train accuracy:  [0.0825]
300 epch loss:  0.23347627    train accuracy:  [0.0955]
350 epch loss:  0.23239844    train accuracy:  [0.095]
400 epch loss:  0.23227236    train accuracy:  [0.119]
test accuracy:  [0.115]
450 epch loss:  0.2294564    train accuracy:  [0.14]
500 epch loss:  0.22905229    train accuracy:  [0.152]
550 epch loss:  0.22907148    train accuracy:  [0.1285]
600 epch loss:  0.23021816    train accuracy:  [0.1305]
test accuracy:  [0.121]
650 epch loss:  0.23129123    train accuracy:  [0.112]
700 epch loss:  0.2315977    train accuracy:  [0.093]
750 epch loss:  0.23202363    t

6050 epch loss:  0.23123375    train accuracy:  [0.1265]
6100 epch loss:  0.22934873    train accuracy:  [0.1575]
6150 epch loss:  0.2289218    train accuracy:  [0.1645]
6200 epch loss:  0.22846186    train accuracy:  [0.1535]
test accuracy:  [0.142]
6250 epch loss:  0.23023058    train accuracy:  [0.135]
6300 epch loss:  0.23086162    train accuracy:  [0.122]
6350 epch loss:  0.2314019    train accuracy:  [0.116]
6400 epch loss:  0.2310524    train accuracy:  [0.108]
test accuracy:  [0.1325]
6450 epch loss:  0.23035732    train accuracy:  [0.0875]
6500 epch loss:  0.23202056    train accuracy:  [0.097]
6550 epch loss:  0.23207876    train accuracy:  [0.128]
6600 epch loss:  0.23113728    train accuracy:  [0.0995]
test accuracy:  [0.136]
6650 epch loss:  0.23048794    train accuracy:  [0.16]
6700 epch loss:  0.22831884    train accuracy:  [0.172]
6750 epch loss:  0.2288039    train accuracy:  [0.149]
6800 epch loss:  0.22950535    train accuracy:  [0.1395]
test accuracy:  [0.1275]
6850

In [ ]:
#关于evaluate部分，在训练过程中可以使用一部分数据集来验证模型的准确率，本程序将验证集合测试集视为相同。

def evaluate(model_path, test_img):
    img = tf.placeholder(tf.float32, shape=(28,28,3))
    
    net = network()
    
    predict = net.test(img)
    predict = tf.nn.softmax(predict)
    
    label = tf.argmax(predcit,1)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        tf.train.Saver(max_to_keep=None).restore(sess, model_path)
        pred, label = sess.run([predict, label], feed_dict={img:test_img})
        
        print(pred, label)


    
    
        

In [10]:
image = './19.bmp'
model_path = os.getcwd() +'/'+'model/model.ckpt'
img = cv2.imread(image)
evaluate(model_path,img)



SyntaxError: can't assign to operator (<ipython-input-10-293a1d9b8c02>, line 1)